---
title: 'Example: Infectious Model'
jupyter: 'julia-1.8'
execute: 
  freeze: auto
---

In [1]:
using Pkg

  Activating new project at `~/Documents/GitHub/Julia-for-SciML/tutorial`


In [2]:
x = 1 + 1

2